Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

---

<!--NOTEBOOK_HEADER-->
*This notebook contains material from [PyRosetta](https://RosettaCommons.github.io/PyRosetta.notebooks);
content is available [on Github](https://github.com/RosettaCommons/PyRosetta.notebooks.git).*

<!--NAVIGATION-->
< [RosettaScripts in PyRosetta](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/02.07-RosettaScripts-in-PyRosetta.ipynb) | [Contents](toc.ipynb) | [Index](index.ipynb) | [Rosetta Energy Score Functions](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/03.00-Rosetta-Energy-Score-Functions.ipynb) ><p><a href="https://colab.research.google.com/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/02.08-Visualization-and-pyrosetta.distributed.viewer.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>

# Visualization and `pyrosetta.distributed.viewer`

*Warning*: This notebook uses `pyrosetta.distributed.viewer` code, which runs in `jupyter notebook` and might not run if you're using `jupyterlab`.

The `pyrosetta.distributed` Viewer quickly renders `.pdb` files, dynamically instantiating `Pose` objects if required for certain visualization modules (matching the name `viewer.set*`). So when adding
visualization modules to the Viewer or using presets, passing `Pose` or `PackedPose` objects to the Viewer is suggested for quicker rendering. If a `Pose` object or `list`, `tuple`, or `set` of `Pose`
objects are provided to the Viewer, the Viewer will dynamically update upon `Pose` conformational changes by calling the `view.show()` method or equivalently `view()`. The Viewer applies visualization modules in the same order they are added (from left to right), so layering different styles (and `ResidueSelectors`) on top of one another becomes possible. The user must have already initialized PyRosetta providing `.params` files for any ligands and non-canonical residues in the input `Pose`(s)/`PackedPose`(s)/`.pdb` file(s), otherwise `pyrosetta.distributed` automatically initializes PyRosetta with default command line options.

Throughout this notebook, different syntaxes for constructing visualizations are demonstrated to show the flexibility of the `pyrosetta.distributed.viewer` API, with hopes you find your favorite.

**If you are using Google Colab:** Currently, `pyrosetta.distributed.viewer` does **not** require the local machine's PyMOL. You can always dump any `Pose` object into a `.pdb` file and open that in PyMOL on your own computer.
```
pose.dump_pdb("output_file.pdb")
```

**Note:** If you are having trouble using this notebook, please use the `PyRosetta.notebooks` conda environment.  Please see chapter 1.0 for instructions on how to setup and use this environment. 

In [ ]:
import glob
import logging
logging.basicConfig(level=logging.INFO)
import numpy as np
import os
import pyrosetta
import pyrosetta.distributed
import pyrosetta.distributed.io as io
import pyrosetta.distributed.viewer as viewer
import sys

if 'google.colab' in sys.modules:
    !pip install pyrosettacolabsetup
    import pyrosettacolabsetup
    pyrosettacolabsetup.setup()
    print ("Notebook is set for PyRosetta use in Colab.  Have fun!")

### Available `viewer` objects:

In [ ]:
viewer.__all__

For example: `viewer.init()`

### Available `viewer.presets` objects:

In [ ]:
viewer.presets.__all__

For example: `viewer.presets.coreBoundarySurface()`

### `viewer` objects contain docstrings as user documentation:

In [ ]:
help(viewer.init)

### `viewer.presets` objects contain docstrings as brief descriptions of each preset visualization

In [ ]:
help(viewer.presets.coreBoundarySurface)

In [ ]:
help(viewer.presets.ligandsAndMetals)

**From previous section:**
Make sure you are in the right directory for accessing the `.pdb` files:

`cd google_drive/My\ Drive/student-notebooks/`

**Note**: `import pyrosetta.distributed.viewer` expands the Jupyter notebook cell width to fit your internet browser, and can be called anytime using `pyrosetta.distributed.viewer.expand_notebook()`

In [ ]:
flags = """
-auto_setup_metals 1
-detect_disulf 1
"""
pyrosetta.distributed.init(flags)

Show basic line representation of `.pdb` file on disk:

>```
view = viewer.init("path/to/my.pdb")
view.show()
```

Change the `window_size` option of `viewer.init()` to fit your screen size, which should prevent the need to scroll to view your macromolecular objects.

Show basic line representation of `Pose` or `PackedPose` objects in memory:

In [ ]:
pose = pyrosetta.io.pose_from_file("inputs/3EK4.pdb")

In [ ]:
view = viewer.init(pose, window_size=(800, 600))
view()

To show basic cartoon representation, add `viewer.setStyle()`:

In [ ]:
view = viewer.init(pose, window_size=(800, 600)) + viewer.setStyle()
view()

To visualize ligands, non-conanical residues, metals, etc., `viewer.setStyle()` (and several other `viewer.set*` objects) optionally accept PyRosetta `ResidueSelector` objects. In the case of `inputs/3EK4.pdb` which is a $Ca^{2+}$-saturated GCaMP2 monomer, a ligand is a $Ca^{2+}$ ion and a non-canonical residue is the chromophore "CRO", both of which are aleady parameterized and available in the PyRosetta residue type set database. Therefore, we do not need to initialize PyRosetta with custom "CA" or "CRO" `.params` files. However, if one wishes to visualize ligands and non-canonical residues for which `.params` files are not available in the PyRosetta residue type set database, one must first initialize PyRosetta providing `.params` files for any ligands and non-canonical residues in the input `Pose`(s) or `PackedPose`(s):

> pyrosetta.init("-extra_res_fa /path/to/LIG.fa.params")

`pyrosetta.distributed.viewer` passes the `Pose`'s PDB numbering of selected residues to `py3dmol` for macromolecular rendering. Also, one may layer any visualization modules with any `ResidueSelectors` in the order they are summed (from left to right). In this case:

In [ ]:
metals_selector = pyrosetta.rosetta.core.select.residue_selector.ResiduePropertySelector(
     pyrosetta.rosetta.core.chemical.ResidueProperty.METAL
)
ligands_selector = pyrosetta.rosetta.core.select.residue_selector.ResiduePropertySelector(
     pyrosetta.rosetta.core.chemical.ResidueProperty.LIGAND
)

view = viewer.init(pose, window_size=(800, 600)) \
    + viewer.setStyle() \
    + viewer.setStyle(residue_selector=ligands_selector, style="stick", colorscheme="magentaCarbon", radius=0.5) \
    + viewer.setStyle(residue_selector=metals_selector, style="sphere", colorscheme="chainHetatm", radius=1.5)

 1. `viewer.setStyle()` layers the cartoon backbone and stick side-chain representation by default
 2. `viewer.setStyle(residue_selector=ligands_selector, style="stick", colorscheme="magentaCarbon", radius=0.5)` layers any residue with the `LIGAND` property a 0.5 radius stick with magenta carbon atoms.
 3. `viewer.setStyle(residue_selector=metals_selector, style="sphere", colorscheme="chainHetatm", radius=1.5)` layers any residue with the `METAL` property a 1.5 radius sphere with the `py3dmol` color scheme "chainHetatm". 

In [ ]:
view()

Show many `Pose`s in memory using the interactive slider widget:

In [ ]:
poses = [pyrosetta.io.pose_from_sequence("TESTVIEWER" * i) for i in range(1, 10)]

In [ ]:
view = viewer.init(poses, continuous_update=True)
view.add(viewer.setStyle(colorscheme="lightgreyCarbon"))
view.add(viewer.setHydrogenBonds(radius=0.1, dashed=False))
view()

Below, layer a `ResidueSelector` of residues with `ResidueProperty.POLAR`, then hydrogen atoms, hydrogen bonds, and disulfide bonds:

In [ ]:
polar_residue_selector = pyrosetta.rosetta.core.select.residue_selector.ResiduePropertySelector(
    pyrosetta.rosetta.core.chemical.ResidueProperty.POLAR
)
# Optionally, initialize the visualizer with a PackedPose object:
# NOTE: PackedPose requires the serialization build of PyRosetta. 
# pose = io.to_packed(pose) # Test the Viewer with a PackedPose object
view = viewer.init(pose)
view.add(viewer.setStyle(radius=0.1))
view.add(viewer.setStyle(residue_selector=polar_residue_selector, colorscheme="whiteCarbon", radius=0.25, label=False))
view.add(viewer.setHydrogens(color="white", polar_only=True, radius=0.1))
view.add(viewer.setHydrogenBonds(color="black"))
view.add(viewer.setDisulfides(radius=0.1))
view()

In [ ]:
view = sum(
    [
        viewer.init(pose),
        viewer.setStyle(cartoon=False, style="sphere", radius=1.5, colorscheme="darkgreyCarbon"),
        viewer.setZoom(factor=0.95)
    ]
)
view()

In [ ]:
pose = pyrosetta.io.pose_from_file("inputs/6MSR.pdb")

In [ ]:
chA = pyrosetta.rosetta.core.select.residue_selector.ChainSelector("A")
chB = pyrosetta.rosetta.core.select.residue_selector.ChainSelector("B")
view = sum(
    [
        viewer.init(pose),
        viewer.setStyle(cartoon_color="lightgrey", radius=0.25),
        viewer.setSurface(residue_selector=chA, colorscheme="greenCarbon", opacity=0.65, surface_type="VDW"),
        viewer.setSurface(residue_selector=chB, color="blue", opacity=0.75, surface_type="SAS"),
        viewer.setDisulfides(radius=0.25),
        viewer.setZoom(factor=1.5)
    ]
)
view()

In [ ]:
view.reinit() # Subtract all visualization modules previously added to the Viewer
view()

`viewer.setZoomTo()` can zoom into a PyRosetta `ResidueSelector`:

In [ ]:
pose = pyrosetta.io.pose_from_file("inputs/1QCQ.pdb")

In [ ]:
helix_selector = pyrosetta.rosetta.core.select.residue_selector.SecondaryStructureSelector("H")
sheet_selector = pyrosetta.rosetta.core.select.residue_selector.SecondaryStructureSelector("E")
loop_selector = pyrosetta.rosetta.core.select.residue_selector.SecondaryStructureSelector("L")

modules = [
    viewer.setBackgroundColor(color="black"),
    viewer.setStyle(residue_selector=helix_selector, cartoon_color="cyan", label=False, radius=0),
    viewer.setStyle(residue_selector=sheet_selector, cartoon_color="red", label=False, radius=0),
    viewer.setStyle(residue_selector=loop_selector, cartoon_color="white", label=False, radius=0),
    viewer.setZoomTo(residue_selector=sheet_selector)
]

In [ ]:
viewer.init(pose, window_size=(1200, 600), modules=modules).show()

View live trajectories:

In [ ]:
pose = pyrosetta.io.pose_from_file("inputs/2FD7.pdb")

In [ ]:
view = viewer.init(pose, delay=0.15) \
    + viewer.setStyle(radius=0.1) \
    + viewer.setDisulfides(radius=0.1)
backrub = pyrosetta.rosetta.protocols.backrub.BackrubMover()
minimize = pyrosetta.rosetta.protocols.minimization_packing.MinMover()

In [ ]:
for _ in range(100):
    backrub.apply(pose)
    minimize.apply(pose)
    view.show()

In [ ]:
pose = pyrosetta.Pose()
view = viewer.init(pose, delay=0.5) + viewer.setStyle(radius=0.1)

add_pdb_info_mover = pyrosetta.rosetta.protocols.simple_moves.AddPDBInfoMover()
mb = pyrosetta.rosetta.protocols.helical_bundle.MakeBundle()
bpc = pyrosetta.rosetta.protocols.helical_bundle.BPC_Parameters

n_helices = 4
for h in range(1, (n_helices + 1)):
    mb.add_helix()
    mb.helix(h).calculator_op().real_parameter(bpc.BPC_delta_omega0).set_value(h * (np.radians(360) / n_helices))
for i in range(2, 42, 3):
    for h in range(1, (n_helices + 1)):
        mb.helix(h).set_helix_length(i)
        mb.helix(h).calculator_op().real_parameter(bpc.BPC_r0).set_value(n_helices * 2)
        mb.helix(h).calculator_op().real_parameter(bpc.BPC_omega0).set_value(np.radians(5))
    mb.apply(pose)
    add_pdb_info_mover.apply(pose)
    view()

In [ ]:
pose = pyrosetta.Pose()
view = viewer.init(pose, delay=0.5) + viewer.setStyle(radius=0.1)

add_pdb_info_mover = pyrosetta.rosetta.protocols.simple_moves.AddPDBInfoMover()
mb = pyrosetta.rosetta.protocols.helical_bundle.MakeBundle()
bpc = pyrosetta.rosetta.protocols.helical_bundle.BPC_Parameters

n_helices = 6
for h in range(1, (n_helices + 1)):
    mb.add_helix()
    mb.helix(h).calculator_op().real_parameter(bpc.BPC_delta_omega0).set_value(h * (np.radians(360) / n_helices))
for i in np.arange(1, 15):
    for h in range(1, (n_helices + 1)):
        mb.helix(h).set_helix_length(20)
        mb.helix(h).calculator_op().real_parameter(bpc.BPC_r0).set_value(i)
        mb.helix(h).calculator_op().real_parameter(bpc.BPC_omega0).set_value(np.radians(4))
    mb.apply(pose)
    add_pdb_info_mover.apply(pose)
    view()

Display preset custom viewers for routine visualizations:

In [ ]:
pdbfiles = [os.path.join("inputs", f) for f in ["3EK4.pdb", "6MSR.pdb", "1QCQ.pdb", "2FD7.pdb"]]
poses = [pyrosetta.io.pose_from_file(p) for p in pdbfiles]

In [ ]:
# Display preset custom viewers for routine visualizations:
viewer.presets.coreBoundarySurface(poses, window_size=(800, 600), continuous_update=True)

In [ ]:
viewer.presets.ligandsAndMetals(poses, window_size=(800, 600), continuous_update=False)

**For Developers**: contribute your custom preset visualizations to PyRosetta:

 1. Edit `~Rosetta/main/source/src/python/PyRosetta/src/pyrosetta/distributed/viewer/presets/\_\_init\_\_.py`
 2. Copy and modify the `templatePreset` function, renaming it to the name of your new preset Viewer (e.g. `myCustomPreset` in step #5).
 3. Add the name of your new preset Viewer to the `__all__` list.
 4. Merge Github pull request into `RosettaCommons/main`
 5. Example using it:
 ```
    import pyrosetta.distributed.viewer as viewer
    viewer.presets.myCustomPreset(poses)
 ```

In [ ]:
def myCustomPreset(packed_and_poses_and_pdbs=None, *args, **kwargs):
    """
    Add a description of the preset Viewer here
    """
    # Add custrom ResidueSelectors
    metals_selector = pyrosetta.rosetta.core.select.residue_selector.ResiduePropertySelector(
         pyrosetta.rosetta.core.chemical.ResidueProperty.METAL
    )
    ligands_selector = pyrosetta.rosetta.core.select.residue_selector.ResiduePropertySelector(
         pyrosetta.rosetta.core.chemical.ResidueProperty.LIGAND
    )
    
    # Add custom Viewer commands
    view = viewer.init(packed_and_poses_and_pdbs=packed_and_poses_and_pdbs, *args, **kwargs) \
        + viewer.setBackgroundColor("white") \
        + viewer.setStyle(style="stick", colorscheme="lightgreyCarbon", radius=0.15) \
        + viewer.setStyle(residue_selector=ligands_selector, style="stick", colorscheme="brownCarbon", radius=0.5, label=True) \
        + viewer.setStyle(residue_selector=metals_selector, style="sphere", colorscheme="chainHetatm", radius=1.5, label=True) \
        + viewer.setHydrogenBonds() \
        + viewer.setDisulfides(radius=0.15) \
        + viewer.setHydrogens(color="white", radius=0.033, polar_only=True) \
        + viewer.setSurface(residue_selector=ligands_selector, surface_type="VDW", opacity=0.5, color="magenta") \
        + viewer.setSurface(residue_selector=metals_selector, surface_type="VDW", opacity=0.5, color="magenta") \
        + viewer.setZoomTo(residue_selector=ligands_selector)

    return view()

myCustomPreset(poses[0])

# Documentation for `pyrosetta.distributed.viewer` visualization modules

In [ ]:
help(viewer.setBackgroundColor)

In [ ]:
help(viewer.setDisulfides)

In [ ]:
help(viewer.setHydrogenBonds)

In [ ]:
help(viewer.setHydrogens)

In [ ]:
help(viewer.setStyle)

In [ ]:
help(viewer.setSurface)

In [ ]:
help(viewer.setZoom)

In [ ]:
help(viewer.setZoomTo)

<!--NAVIGATION-->
< [RosettaScripts in PyRosetta](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/02.07-RosettaScripts-in-PyRosetta.ipynb) | [Contents](toc.ipynb) | [Index](index.ipynb) | [Rosetta Energy Score Functions](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/03.00-Rosetta-Energy-Score-Functions.ipynb) ><p><a href="https://colab.research.google.com/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/02.08-Visualization-and-pyrosetta.distributed.viewer.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>